<a href="https://colab.research.google.com/github/sharmaamey/demoproject/blob/main/Evaluserve_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [62]:
import pandas as pd
import numpy as np
import datetime


Data input function

In [63]:


# Step 2: Load Input Data
def load_input_data(file_path):
    df = pd.read_excel(file_path, sheet_name="Strategy")
    holidays = pd.read_excel(file_path, sheet_name="Holidays")
    return df, holidays

Data cleaning preprocessing

In [64]:
# Clean and process data

def clean_data(data, holidays):


    data.columns = data.iloc[0]

    # Drop the first row since it's now the column names
    data = data[1:]

    # Reset the index if needed
    data.reset_index(drop=True, inplace=True)
    data.dropna(inplace=True)

    holidays = holidays[['London Holidays']]


    # Assuming 'Date' is the datetime column
    data['Date'] = pd.to_datetime(data['Date']).copy()
    data['Date'] = data['Date'] = data['Date'].dt.date.copy()

    # Now, 'Date' column only contains the date portion

    return data, holidays



Function to caqlculate index levels and store final output and intermediate output files

In [65]:
#Function for Index Levels


def index_level(main_df, holidays, underlying_data, J0, t0, n, output_csv_filename):
    # ... (the rest of your code)
     # Initliasing variables for calculations
    count = 0
    prev_month = None
    printed_month = None
    R_date = t0
    last_date = t0
    Jr = J0
    result_dates = []
    result_Jt = []

    main_df = main_df[main_df['Date'] >= t0]

    # Extract London Holidays from the Holidays DataFrame
    london_holidays = holidays['London Holidays'].tolist()

    # Create a list to store intermediate results
    intermediate_results = []

    # Iterate through the DataFrame
    for index, row in main_df.iterrows():
        t_date = row['Date']  # Date for index calculation
        m = (t_date - last_date).days if last_date else 1
        last_date = t_date

        # Check if t_date is a London holiday
        if t_date in london_holidays:
            # If it's a holiday, skip this date for index calculation
            continue

        # Check if t_date's month is in [1, 4, 7, 10]
        if t_date.month in [1, 4, 7, 10]:
            current_month = t_date.month

            # If a new month is encountered, reset count and prev_month
            if current_month != prev_month:
                count = 1  # Start counting from 1 for the new month
                prev_month = current_month
                printed_month = None  # Reset printed_month
            else:
                count += 1

            # Check if count is 8 and it's the first time in this month
            if count == 8 and printed_month != current_month:
                # Calculate new Jr
                Jx = Jr * (1 + np.sum([underlying_data[f'Underlying{k}']["Weight"] *
                                       (main_df.loc[main_df['Date'] == t_date][f'Underlying{k}'].values[0] /
                                        main_df.loc[main_df['Date'] == R_date][f'Underlying{k}'].values[0] - 1)
                                       for k in range(1, n + 1)])) * (1 - np.sum([underlying_data[f'Underlying{k}']["Rebalance Cost"] * m / 220
                                                                                  for k in range(1, n + 1)]))

                # Update Jr
                Jr = Jx
                # Update R_date
                R_date = t_date
                # Update variables at the end of statements
                printed_month = current_month
                count = 0



        # Calculate Jt
        if t_date == t0:
            Jt = J0
            result_dates.append(t_date)
            result_Jt.append(Jt)
        else:
            Jt = Jr * (1 + np.sum([
                underlying_data[f'Underlying{k}']["Weight"] *
                (main_df.loc[main_df['Date'] == t_date][f'Underlying{k}'].values[0] /
                 main_df.loc[main_df['Date'] == R_date][f'Underlying{k}'].values[0] - 1)
                for k in range(1, n + 1)
            ])) * (1 - np.sum([
                underlying_data[f'Underlying{k}']["Rebalance Cost"] * m / 220
                for k in range(1, n + 1)
            ]))

            # Add Jt
            result_dates.append(t_date)
            result_Jt.append(Jt)
            # Create a dictionary to store intermediate values for this date
            intermediate_values = {
                'Date': t_date,
                'Intermediate Refernce Index Level': Jr,
                'Index Level': Jt,
                'Number of days': m
            }

            # Collect the values used in Jt calculation for each 'k'
            for k in range(1, n + 1):
                weight = underlying_data[f'Underlying{k}']["Weight"]
                value_t = main_df.loc[main_df['Date'] == t_date][f'Underlying{k}'].values[0]
                value_R = main_df.loc[main_df['Date'] == R_date][f'Underlying{k}'].values[0]
                rebalance_cost = underlying_data[f'Underlying{k}']["Rebalance Cost"]

                intermediate_values[f'Weight_{k}'] = weight
                intermediate_values[f'Underlying_t_{k}'] = value_t
                intermediate_values[f'Underlying_R_{k}'] = value_R
                intermediate_values[f'Rebalance_Cost_{k}'] = rebalance_cost

            # Append the intermediate values to the list
            intermediate_results.append(intermediate_values)




    # Create a DataFrame with Jt and dates
    result_df = pd.DataFrame({'Date': result_dates, 'Index Levels': result_Jt})

    # Save the final result to a CSV file
    result_df.to_csv(output_csv_filename, index=False)

    # Save intermediate results to a separate CSV file
    intermediate_df = pd.DataFrame(intermediate_results)
    intermediate_df.to_csv('intermediate_results.csv', index=False)

    return result_df


In [66]:


def calculate_annualized_returns(index_df, tenors):
    annualized_returns = {}

    # Calculate ILDtt and ILt (latest values)
    ILDtt = index_df['Index Levels'].iloc[-1]
    ILt = ILDtt  # ILt is the same as ILDtt on the latest day

    for tenor in tenors:
        if tenor == 'All':
            # For 'All', use the entire dataset
            ILDtt_nY = index_df['Index Levels'].iloc[0]
        else:
            # Calculate ILDtt-nY based on the tenor
            nY = int(tenor.replace('Year', ''))
            ILDtt_nY_index = max(0, len(index_df) - nY * 252)
            ILDtt_nY = index_df['Index Levels'].iloc[ILDtt_nY_index]

        # Calculate the annualized return based on the formula
        annualized_return = ((ILt / ILDtt_nY) ** (365.25 / len(index_df))) - 1

        annualized_returns[tenor] = annualized_return * 100  # Convert to percentage
        annualized_returns_df = pd.DataFrame.from_dict(annualized_returns, orient="index", columns=["Annualized Return (%)"])

    return annualized_returns_df

#annualized_returns = calculate_annualized_returns(index_df, tenors)


Function to calculate annualised risk

In [67]:
def calculate_annualized_risk(index_df, tenors):
    # Calculate one-day return series: (ILt / ILt-1) - 1
    index_df['Return'] = index_df['Index Levels'].pct_change()

    annualized_risks = {}

    for tenor in tenors:
        if tenor == 'All':
            # For 'All', use the entire return series
            returns = index_df['Return']
        else:
            # Calculate the number of days based on the tenor
            nY = int(tenor.replace('Year', ''))
            days = nY * 252  # Assuming 252 trading days in a year

            # Use the last 'days' of return data for the specified tenor
            returns = index_df['Return'].tail(days)

        # Calculate annualized standard deviation for the tenor
        annualized_risk = returns.std() * (252 ** 0.5)

        annualized_risks[tenor] = annualized_risk
        annualized_risks_df = pd.DataFrame.from_dict(annualized_risks, orient="index", columns=["Annualized Risk"])

    return annualized_risks_df


#annualized_risks = calculate_annualized_risk(index_df, tenors)


main function

In [68]:


def main():
    file_path = "data.xlsx"
    data, holidays = load_input_data(file_path)
    main_df, holiday = clean_data(data, holidays)

    # Weights and rebalancing fractions
    underlying_data = {
        "Underlying1": {"Weight": 0.1, "Rebalance Cost": 0.005},
        "Underlying2": {"Weight": 0.2, "Rebalance Cost": 0.0025},
        "Underlying3": {"Weight": 0.15, "Rebalance Cost": 0.001},
        "Underlying4": {"Weight": 0.05, "Rebalance Cost": 0.001},
        "Underlying5": {"Weight": 0.3, "Rebalance Cost": 0.0025},
        "Underlying6": {"Weight": 0.2, "Rebalance Cost": 0.0025},
    }

    J0 = 100  # base index level
    t0 = datetime.datetime(2021, 1, 12).date()  # base date
    Jr = J0  # initializing Jr as J0
    n = 6  # Number of underlyings

    tenors = ['1Year', '2Year', '3Year', '5Year', 'All']

    # Run index_level function and pass its result to the next functions
    index_df = index_level(main_df, holiday, underlying_data, J0, t0, n, 'output.csv')

    # Run calculate_annualized_risk function and pass its result to the next function
    annualized_risks = calculate_annualized_risk(index_df, tenors)

    # Run calculate_annualized_returns function
    annualized_returns = calculate_annualized_returns(index_df, tenors)


    print(annualized_risks)
    print(annualized_returns)

if __name__ == "__main__":
    main()


<ipython-input-64-e5debbe51cc8>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)
<ipython-input-64-e5debbe51cc8>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'] = pd.to_datetime(data['Date']).copy()
<ipython-input-64-e5debbe51cc8>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  da

       Annualized Risk
1Year         1.858339
2Year         1.830042
3Year         1.830042
5Year         1.830042
All           1.830042
       Annualized Return (%)
1Year              14.446726
2Year              48.690408
3Year              48.690408
5Year              48.690408
All                48.690408
